<h1>Detecting Languages Programmatically</h1>
<p>We have been detecting languages as a species for quite some time now, and there are numerous tools that exist (such as nltk) that allow us to do this programmatically. In this notebook, we will be taking a different approach, developing a language detector from scratch using a combination of simple algorithms. Later on, we will incorporate some of the more advanced tools that exist to see how they compare to our own implementation.

When preparing for this notebook, I ended up creating far more tools than I had intended, and then condensing them down into some more robust, simpler functions as I realized the direction I wanted to take this project. Regardless, what that means for us now is that we have several functions we will use to produce some general information about the language from which we are (currently) assuming our plaintexts originate.

We are going to start out by counting a lot of things to generate some information that is a bit harder to find across the internet. For example, I was able to locate the top 20 $n$-grams from $n=2, 3, 4$, but I would like more than just 20. We are going to construct $26\times 26\times \cdots \times 26$ arrays where the $i_1,i_2,\dots,i_n$ element is equal to the relative frequency of the $n$-gram consisting of letters $i_1, i_2, \cdots, i_n$ where each $i_j\in\{a, b, \cdots, z\}$ for $j\in\{1, 2, \dots, n\}$. </p>